## Import Libraries

In [221]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTENC

from xgboost import XGBClassifier
from sklearn.ensemble import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import *

## Import Data

In [222]:
# Reading Processed Train Set Data File
raw_d1 = pd.read_csv('Data/Processed_Trainset.csv')

# Feature Headers List
headers = list(raw_d1.columns)

# Reading Test Data Set
raw_d2 = pd.read_csv('Data/Processed_Testset.csv')

id_col = list(raw_d2.Client_ID)
raw_d2.drop(['Client_ID'], axis=1, inplace=True)

## Correlation

In [223]:
# Generating Heat Map
correlation = raw_d1.corr()
correlation.style.background_gradient(cmap='coolwarm')
# Removed Highly Correlated features using this heatmap

,bl_ 500K,bl_1.5M,bl_100K,bl_1M,bl_2.5M,bl_200K,bl_300K,bl_400K,g_F,g_M,es_Graduate,es_High School,es_Other,ms_Other,ms_Single,age_31-45,age_46-65,age_Less than 30,age_More than 65,Paid_Due,Pay,NEXT_MONTH_DEFAULT
bl_ 500K,1,-0.0809181,-0.148581,-0.208257,-0.0142515,-0.189785,-0.121203,-0.138775,-0.0316169,0.0316169,0.0686536,-0.0391123,-0.0367187,-0.0220277,0.0220277,0.0693133,-0.0269624,-0.0552351,0.0210128,0.012179,-0.111011,-0.0370965
bl_1.5M,-0.0809181,1,-0.0913996,-0.12811,-0.00876686,-0.116747,-0.0745584,-0.0853675,0.0270994,-0.0270994,0.107875,-0.0504154,-0.065868,-0.0459557,0.0459557,0.0671006,0.0364036,-0.104032,0.0189018,0.178042,-0.0887099,-0.0541162
bl_100K,-0.148581,-0.0913996,1,-0.235233,-0.0160976,-0.214368,-0.136903,-0.15675,0.0803429,-0.0803429,-0.146208,0.0911873,0.0723564,0.0712496,-0.0712496,-0.136678,0.0215493,0.131416,-0.0169915,-0.198443,0.229785,0.136035
bl_1M,-0.208257,-0.12811,-0.235233,1,-0.0225631,-0.300469,-0.191889,-0.219709,-0.0237629,0.0237629,0.16509,-0.093457,-0.0887376,-0.0710705,0.0710705,0.145466,-0.0102127,-0.14892,0.00826879,0.161617,-0.21914,-0.100197
bl_2.5M,-0.0142515,-0.00876686,-0.0160976,-0.0225631,1,-0.0205618,-0.0131314,-0.0150352,0.00714508,-0.00714508,0.015397,-0.00876073,-0.0082431,-0.0024515,0.0024515,0.0070284,0.0101155,-0.0154475,-0.00238377,0.0536447,-0.0151615,-0.0133301
bl_200K,-0.189785,-0.116747,-0.214368,-0.300469,-0.0205618,1,-0.174869,-0.200221,0.0172643,-0.0172643,-0.13781,0.0807053,0.0720818,0.0422606,-0.0422606,-0.118711,0.0225088,0.110434,-0.00992841,-0.123155,0.15785,0.0508243
bl_300K,-0.121203,-0.0745584,-0.136903,-0.191889,-0.0131314,-0.174869,1,-0.127868,-0.0386247,0.0386247,-0.0396368,0.0103811,0.0302291,0.00556912,-0.00556912,-0.0243842,-0.0120591,0.0371067,-0.00880676,-0.00588913,0.0586455,0.0235289
bl_400K,-0.138775,-0.0853675,-0.15675,-0.219709,-0.0150352,-0.200221,-0.127868,1,-0.0278914,0.0278914,0.00895552,-0.0129123,0.000990892,0.010742,-0.010742,0.0133994,-0.0247546,0.00631533,-0.00462312,0.0249757,-0.0402066,-0.025435
g_F,-0.0316169,0.0270994,0.0803429,-0.0237629,0.00714508,0.0172643,-0.0386247,-0.0278914,1,-1,0.0213745,0.0100856,-0.0279094,0.0315728,-0.0315728,0.0244727,0.0604214,-0.0778864,0.0197493,0.0272415,0.0648831,0.0416493
g_M,0.0316169,-0.0270994,-0.0803429,0.0237629,-0.00714508,-0.0172643,0.0386247,0.0278914,-1,1,-0.0213745,-0.0100856,0.0279094,-0.0315728,0.0315728,-0.0244727,-0.0604214,0.0778864,-0.0197493,-0.0272415,-0.0648831,-0.0416493


## Formatting

In [224]:
# Convert Data into ndArrays
train_data = np.array(raw_d1)
test_data = np.array(raw_d2)

# Total Training Data
X_tot = train_data[:, :-1]
Y_tot = train_data[:, -1]

# Splitting Data into a Training Set and a Cross Validation Set
X_train, X_cross, Y_train, Y_cross = train_test_split(X_tot, Y_tot, test_size = 0.15, random_state = 42, shuffle = True)
X_train.shape, Y_train.shape

((20400, 21), (20400,))

## SMOTE

In [225]:
# smote = SMOTENC(list(range(0,18)), sampling_strategy='auto', k_neighbors=10, n_jobs=1)
# X_train,Y_train = smote.fit_resample(X_train, Y_train)

# pd.DataFrame(np.concatenate((X_train,np.reshape(Y_train, (Y_train.shape[0], 1))), axis=1)
#              ,columns=headers).to_csv('Data/SMOTED.csv', index=None)
# X_train.shape, Y_train.shape

## Normalizing Data

In [226]:
# sc = RobustScaler()
# sc.fit(X_tot)

# X_tot = sc.fit_transform(X_tot)
# X_train = sc.transform(X_train)
# X_cross = sc.transform(X_cross)
# test_data = sc.transform(test_data)

# pd.DataFrame(X_tot, columns=headers[:-1])

## Classification Model 

In [230]:
rf = GradientBoostingClassifier(n_estimators= 300)

# rf = RandomForestClassifier(bootstrap=True, class_weight= None, max_depth= 8, max_features= 10, max_leaf_nodes= None,
#                             min_samples_leaf= 1, min_samples_split= 2, min_weight_fraction_leaf= 0.0, n_estimators= 500,
#                             n_jobs= 1, oob_score= False, random_state= 85, verbose= 0, warm_start= False)
# rf = XGBClassifier()

rf.fit(X_train, Y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=300,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

## Prediction and Evaluation

In [231]:
# Predicting Outcomes
predict_train = rf.predict(X_train)
predict_cross = rf.predict(X_cross)
predict_tot = rf.predict(X_tot)
predict_test = rf.predict(test_data)

# Computing Confusion Matrix on Cross Validation Set
confusion = confusion_matrix(Y_cross,predict_cross)

# Computing Accuracy Values
accuracy_train = accuracy_score(Y_train, predict_train)
accuracy_cross = accuracy_score(Y_cross, predict_cross)
accuracy_tot = accuracy_score(Y_tot, predict_tot)

# Computing AUC Values
auc_train = roc_auc_score(Y_train, predict_train)
auc_cross = roc_auc_score(Y_cross, predict_cross)
auc_tot = roc_auc_score(Y_tot, predict_tot)                

# Printing Accuracy Metrics
print("Train Accuracy            : {:.3f}%       AUC: {:.4f}".format(accuracy_train * 100, auc_train))
print("Cross Validation Accuracy : {:.3f}%       AUC: {:.4f} ".format(accuracy_cross * 100, auc_cross))
print("Total Set Accuracy        : {:.3f}%       AUC: {:.4f}".format(accuracy_tot * 100, auc_tot))
print("Confusion Matrix for Cross Validation Set :\n %s" % str(confusion))
print("Number of Ones Predicted for Test Data : %d" % np.sum(predict_test))

print(classification_report(Y_cross, predict_cross))

Train Accuracy            : 82.809%       AUC: 0.6677
Cross Validation Accuracy : 82.722%       AUC: 0.6562 
Total Set Accuracy        : 82.796%       AUC: 0.6660
Confusion Matrix for Cross Validation Set :
 [[2702  122]
 [ 500  276]]
Number of Ones Predicted for Test Data : 536
              precision    recall  f1-score   support

         0.0       0.84      0.96      0.90      2824
         1.0       0.69      0.36      0.47       776

    accuracy                           0.83      3600
   macro avg       0.77      0.66      0.68      3600
weighted avg       0.81      0.83      0.80      3600



## Upload

In [229]:
# Uploading Predictions to a CSV
df = pd.DataFrame(predict_test, dtype=int)
df.index = id_col
df.to_csv('Data/Submission.csv', header=['NEXT_MONTH_DEFAULT'], index_label='Client_ID')